# Synthetic Feeder, Chennai


In this tutorial we are going to create a synthetic distribution feeder model by leveraging OpenStreet buildings as well as road data in from Chennai, India.

1. [Generating power system loads from Geometry objects](#load_to_geometry)
2. [Using clustering algorithms to create distribution transformers to serve loads](#dist_trans)
3. [ Generating high tension acyclic undirected network to connect distribution transformers using road network from OpenStreet data](#primary_network)
4. [Generating HT lines from primary network](#ht_lines)
5. [Generating secondary network](#lt_lines)

Developed by: Kapil Duwadi (Kapil.Duwadi@nrel.gov), 2022-03-16


Let's get the styling out of the way first. I like to see the width little bit bigger than what is there by default.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

<a name="load_to_geometry"></a>
### 1. Generating power system loads from Geometry objects
Let's get all the buildings from chennai India and print total number of builidings along with sample building.

In [ ]:
from shift.geometry import BuildingsFromPlace
g = BuildingsFromPlace("Chennai, India", max_dist=100)
geometries = g.get_geometries()
print(f"Total number of buildings: {len(geometries)}" +
f"\nSample geometry: {geometries[0]}")

In order to create power system loads from geometries we need to tell how to assign phases, voltages and connection type for loads. In this case let's use RandomPhaseAllocator class to allocate phases to all geometries. Here we are saying all geometries are of single phase type and there are no two phase and three phase loads and finally pass all the geometries.

Similarly we initialize simple voltage setter by passing line to line voltage of 13.2 kV. DefaultConnSetter class is created which will set the connection type to 'wye' for all geometries.

In [ ]:
from shift.load_builder import (RandomPhaseAllocator, 
                                SimpleVoltageSetter, DefaultConnSetter)
rpa = RandomPhaseAllocator(100, 0, 0, geometries)
svs = SimpleVoltageSetter(13.2)
dcs = DefaultConnSetter()

But wait how do we get power consumption data for the load. In order get consumption let's use building area info to get the kw.
Let's say we know there is a piecewise linear function that relates building area with peak kW consumption.

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

area_to_kw_curve = [(0,5), (10, 5.0), (20, 18), (50, 30)]

df = pd.DataFrame({
    'Area in meter square' : [x[0] for x in area_to_kw_curve],
    'Peak consumption in kW' : [x[1] for x in area_to_kw_curve]
})
fig = px.line(df, x="Area in meter square", y="Peak consumption in kW") 
fig.show()

In order to use this piecewise linear function we can invoke PiecewiseBuildingAreaToConsumptionConverter class from load_builder and pass it as an argument to build the load. Let's see this class in action.


In [ ]:
from shift.load_builder import PiecewiseBuildingAreaToConsumptionConverter
pbacc = PiecewiseBuildingAreaToConsumptionConverter(area_to_kw_curve)
area = 15
print(f"For area of {area} m^2 the consumption would be {pbacc.convert(area)}")

Let's build the loads from the geometries and print one them. Here let's try to build constant power factor of 1.0 type loads for simplicity.


In [ ]:
from shift.load_builder import ConstantPowerFactorBuildingGeometryLoadBuilder
from shift.load_builder import LoadBuilderEngineer
loads = []
for g in geometries:
    builder = ConstantPowerFactorBuildingGeometryLoadBuilder(g, 
                            rpa, pbacc, svs, dcs, 1.0)
    b = LoadBuilderEngineer(builder)
    loads.append(b.get_load())
print(len(loads), loads[0], loads[1])

Let's visualize these loads on top of GIS map. That would be cool right ?. In order to do that let's invoke two layer distribution network first.

<script src="https://cdn.plot.ly/plotly-2.12.1.min.js"></script>

In [ ]:
from shift.feeder_network import (SimpleTwoLayerDistributionNetworkBuilder, 
                                  TwoLayerNetworkBuilderDirector)
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
network = TwoLayerNetworkBuilderDirector(loads, [], [], [], network_builder)

In [ ]:
from shift.network_plots import PlotlyGISNetworkPlot
from shift.constants import PLOTLY_FORMAT_CUSTOMERS_ONLY

API_KEY = None
p = PlotlyGISNetworkPlot(
        network.get_network(),
        API_KEY,
        'carto-darkmatter',
        asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_ONLY
    )
p.show()

<a name="dist_trans"></a>
### 2. Using clustering algorithms to create distribution transformers to serve loads
Next step is to use clustering algorithm to figure out best location for positioning distribution transformers. Kmeans clustering is one way of doing this. Let's say we want to have 20 disribution transformers for this feeder.

In [ ]:
from shift.clustering import KmeansClustering
import numpy as np

x_array = np.array([[load.longitude, load.latitude] 
                    for load in loads])
# Try to use kW, along with location information for clustering 
cluster_ = KmeansClustering(2)
clusters = cluster_.get_clusters(x_array)
#print(f'Optimal number of clusters: {cluster_.optimal_clusters}')
#cluster_.plot_scores()
cluster_.plot_clusters()

Let's use clustering algorithm to position distribution transformers. In order to create transformers from clusters we need to provide list of loads, clustering object, kV and connection type for both high tension and low tension side of transformers along with number of phase to be used to design transformers. The diversity factor function, power factor and adjustment_factor is used to compute kVA capacity for the transformer. The catalog is used to find nearest transformer capacity. Let's print the catalog and transformers.

In [ ]:
from shift.transformer_builder import (
    ClusteringBasedTransformerLoadMapper)
from shift.enums import (TransformerConnection, 
                         NumPhase, Phase)

# Planned years of operation
# Actual years of operation
# Assumed forecasted annual load growth
# Actual annual growth rate
# Adjustment factor

trans_builder = ClusteringBasedTransformerLoadMapper(
    loads,
    clustering_object = cluster_,
    diversity_factor_func = lambda x: 0.3908524*np.log(x) + 1.65180707,
    ht_kv = 13.2,
    lt_kv = 0.4,
    ht_conn = TransformerConnection.DELTA,
    lt_conn = TransformerConnection.STAR,
    ht_phase = Phase.ABC,
    lt_phase = Phase.ABCN,
    num_phase = NumPhase.THREE,
    power_factor=0.9,
    adjustment_factor=1.15
)
t  = trans_builder.get_transformer_load_mapping()

Let's visualize the loads and transformers in GIS map.

In [ ]:
from shift.constants import (
    PLOTLY_FORMAT_CUSTOMERS_AND_DIST_TRANSFORMERS_ONLY)
network = TwoLayerNetworkBuilderDirector(loads, 
            list(t.keys()), [], [], network_builder)
p = PlotlyGISNetworkPlot(
        network.get_network(),
        API_KEY,
        'carto-darkmatter',
        asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_AND_DIST_TRANSFORMERS_ONLY
    )
p.show()

<a name="primary_network"></a>
### 3. Generating high tension acyclic undirected network to connect distribution transformers using road network from OpenStreet data
Let's try to get road network from chennai india. We will convert the road network to undirected graph and use minimum spanning tree to remove any loops. 

In [ ]:
from shift.graph import RoadNetworkFromPlace
from shift.constants import PLOTLY_FORMAT_SIMPLE_NETWORK
graph = RoadNetworkFromPlace('chennai, india', max_dist=100)
graph.get_network()
p = PlotlyGISNetworkPlot(
            graph.updated_network,
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()

Let's create an instance of Primary Network Builder class and visulaize the generated network.

In [ ]:
from shift.primary_network_builder import PrimaryNetworkFromRoad
pnet = PrimaryNetworkFromRoad(
        graph,
        t,
        (80.2786311, 13.091658),
        lambda x: 0.3908524*np.log(x) + 1.65180707,
        13.2,
        100
    )
pnet.update_network_with_ampacity()
p = PlotlyGISNetworkPlot(
            pnet.get_sliced_graph(),
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()
p = PlotlyGISNetworkPlot(
            pnet.get_primary_network(),
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()

Let's try to plot ampacity of primary line section versus how far it is from the substation.

<a name="ht_lines"></a>
### 4. Generating HT lines from primary network



In [ ]:
from shift.primary_network_builder import PrimarySectionsBuilder
from shift.enums import ConductorType, NumPhase
from shift.line_section import HorizontalThreePhaseConfiguration
from shift.feeder_network import update_transformer_locations
psections = PrimarySectionsBuilder(
        pnet.get_network(),
        ConductorType.OVERHEAD,
        HorizontalThreePhaseConfiguration(9, 0.4, 'm'),
        NumPhase.THREE,
        False
    )
longest_length = pnet.get_longest_length_in_kvameter() / 1609.34
k_drop = 2 / (longest_length)
l_sections = psections.generate_primary_line_sections(k_drop, 11.0)
r_nodes = pnet.get_trans_node_mapping()
t = update_transformer_locations(r_nodes, t, l_sections)

In [ ]:
from shift.constants import PLOTLY_FORMAT_CUSTOMERS_DIST_TRANSFORMERS_HT_LINE
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
n = TwoLayerNetworkBuilderDirector(
    loads, 
    list(t.keys()),
    l_sections,
    [],
    network_builder)
p = PlotlyGISNetworkPlot(
    n.get_network(),
    API_KEY,
    'light',
    asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_DIST_TRANSFORMERS_HT_LINE
)
p.show()

<a name="lt_lines"></a>
### 5. Generating secondary network

In [ ]:
from shift.secondary_network_builder import SecondaryNetworkBuilder, SecondarySectionsBuilder
s_sections = []
for trans, cust_list in t.items():
    sn = SecondaryNetworkBuilder(cust_list, trans, lambda x: 0.3908524*np.log(x) + 1.65180707, 0.4)
    sn.update_network_with_ampacity()
    sc = SecondarySectionsBuilder(
        sn.get_network(),
        ConductorType.OVERHEAD,
        HorizontalThreePhaseConfiguration(9, 0.4, 'm'),
        HorizontalThreePhaseConfiguration(-1, 0.4, 'm'),
        NumPhase.THREE,
        True
    )
    s_sections.extend(sc.generate_secondary_line_sections())

In [ ]:
from shift.constants import PLOTLY_FORMAT_ALL_ASSETS
from shift.enums import NetworkAsset
PLOTLY_FORMAT_ALL_ASSETS['nodes'][NetworkAsset.DISTXFMR]['color'] = 'red'
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
n = TwoLayerNetworkBuilderDirector(
    loads, 
    list(t.keys()),
    l_sections,
    s_sections,
    network_builder)
p = PlotlyGISNetworkPlot(
    n.get_network(),
    None,
    'carto-darkmatter',
    asset_specific_style=PLOTLY_FORMAT_ALL_ASSETS
)
p.show()